In [1]:
import sys
from pandas.core.frame import DataFrame
import requests
import datetime as dt
from meteostat import Point
from meteostat import Daily 
import matplotlib.pyplot as plt

cities_of_interest = {"Akureyri": "IS", "London": "GB", 
    "Mexico City": "MX", "Newcastle": "GB", "Reykjavík":"IS"}

input = input(f"Please enter one of {cities_of_interest} \n")

if input not in cities_of_interest.keys():
    raise Exception("Bad user, give me the city")

def filter_results_to_country(geoapi_response_data, city):
    #If there are multiple cities with the same name, we choose the most populated
    filter_by_country = [resp for resp in geoapi_response_data if resp['country_code'] == cities_of_interest[city]]
    # 
    sorted_by_pop = sorted(filter_by_country, key = lambda resp: resp['population'] if 'population' in resp else 0, reverse=True)
    result = sorted_by_pop[0]

    # Relevant subset of the result dict
    return {key: result[key] for key in ('latitude', 'longitude', 'elevation', 'population')}

def query_lat_long(city):
    params_dict = {
        'name': city,
        #Default is 10 but since newcastle gives 9, we might hit the limit
        'count': 100
    }

    # Since the user gives us more than 3 chars, the api performs fuzzy matching. So we do not
    # need to worry abt spelling
    resp = requests.get('https://geocoding-api.open-meteo.com/v1/search', params_dict)
    data = resp.json()
    
    return filter_results_to_country(data['results'], city)

def query_historical_weather(lat_long_elevation):
    now = dt.datetime.now()
    three_year_weeks = 52*3
    three_year_delta = dt.timedelta(weeks=three_year_weeks)
    five_days_ago = now - three_year_delta 
    location = Point(
        lat_long_elevation['latitude'], 
        lat_long_elevation['longitude'], 
        lat_long_elevation['elevation'])

    daily = Daily(location, start=five_days_ago, end=now)
    
    # Ask meteostat to fill in any gaps in the data
    daily.normalize()
    data = daily.fetch()
    
    #tavg=Temp average (C).prcp=Total precipitation(mm). wdir=Wind direction(degrees)
    #wspd=Average wind speed(km/h).wpgt=Wind peak gust(km/hr). pres=Sea-level air pressure(hpa)
    return data[['tavg', 'prcp', 'wdir', 'wspd', 'wpgt', 'pres']]

def get_weather(city_country):
    coords = query_lat_long(city_country)
    return query_historical_weather(coords)

data = get_weather(input)
print(data)

Please enter one of {'Akureyri': 'IS', 'London': 'GB', 'Mexico City': 'MX', 'Newcastle': 'GB', 'Reykjavík': 'IS'} 
London
            tavg  prcp   wdir  wspd  wpgt    pres
time                                             
2018-11-28  11.2   3.6  188.4  22.1  51.8  1006.0
2018-11-29  11.5   7.8  202.0  26.4  57.4  1001.2
2018-11-30   8.8   0.0  224.9  16.6  37.0  1006.8
2018-12-01  10.1   6.5  198.1  15.2  33.3  1003.1
2018-12-02  12.7   4.1  233.2  20.0  38.9   997.2
...          ...   ...    ...   ...   ...     ...
2021-11-19  10.3   0.0  243.4   9.9  22.2  1030.5
2021-11-20   8.9   0.0  244.7   8.4  20.4  1024.2
2021-11-21   4.9   0.0  340.9  18.1  40.8  1019.6
2021-11-22   4.3   0.0    6.9  15.4  31.5  1029.8
2021-11-23   3.7   0.0  356.0   7.7  20.4  1032.0

[1092 rows x 6 columns]


In [5]:
# Now we want to get it into a file 
#data.index.name = 'Date'
#data.reset_index(inplace=True)
#data.rename(columns={"time": 'Date'}, inplace=True)


#data.head()
Path = f'Data_weather/London.csv'
data.to_csv(Path)